In [ ]:
!pip install transformers
!pip install datasets
!pip install seqeval


In [ ]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification, create_optimizer, DataCollatorForTokenClassification, keras_callbacks
from datasets import load_dataset, load_metric
import tensorflow as tf
import numpy as np
from transformers.keras_callbacks import KerasMetricCallback
from tensorflow.keras.callbacks import TensorBoard

In [ ]:
task = "ner"
model_checkpoint = 'distilbert-base-uncased'
batch_size = 64
datasets = load_dataset("conll2003")

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [ ]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


In [ ]:
label_all_tokens = True

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)


id2label = {i: label for i, label in enumerate(label_list)}
label2id = {label: i for i, label in enumerate(label_list)}


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForTokenClassification.from_pretrained(
    model_checkpoint, num_labels=len(label_list), id2label=id2label, label2id=label2id
)

In [ ]:
num_train_epochs = 3
num_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs
optimizer, lr_schedule = create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

model.compile(optimizer=optimizer)

In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer, return_tensors="np")

train_set = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)

validation_set = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)


In [ ]:
metric = load_metric("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


metric_callback = KerasMetricCallback(
    metric_fn=compute_metrics, eval_dataset=validation_set
)

tensorboard_callback = TensorBoard(log_dir="./tc_model_save/logs")

callbacks = [metric_callback, tensorboard_callback]

In [ ]:
model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,

)

Epoch 1/3
219/219 [==============================] - 115s 489ms/step - loss: 0.3998 - val_loss: 0.1223 - precision: 0.8061 - recall: 0.8527 - f1: 0.8287 - accuracy: 0.9664
Epoch 2/3
219/219 [==============================] - 95s 435ms/step - loss: 0.1000 - val_loss: 0.0796 - precision: 0.8807 - recall: 0.9019 - f1: 0.8912 - accuracy: 0.9770
Epoch 3/3
219/219 [==============================] - 100s 455ms/step - loss: 0.0694 - val_loss: 0.0735 - precision: 0.8901 - recall: 0.9134 - f1: 0.9016 - accuracy: 0.9787


In [ ]:
sample_sentence = """In the bustling city of Paris, John Doe, a renowned scientist,
                     discussed climate change with President Smith at the Eiffel Tower,
                     emphasizing the need for urgent action."""


tokenized = tokenizer([sample_sentence], return_tensors="np")

In [ ]:
outputs = model(tokenized).logits

classes = np.argmax(outputs, axis=-1)[0]

output_results = [(tokenizer.decode(token), model.config.id2label[id]) for token, id in zip(tokenized["input_ids"][0], classes)]
print(output_results)

[('[CLS]', 'O'), ('in', 'O'), ('the', 'O'), ('bust', 'O'), ('##ling', 'O'), ('city', 'O'), ('of', 'O'), ('paris', 'B-LOC'), (',', 'O'), ('john', 'B-PER'), ('doe', 'I-PER'), (',', 'O'), ('a', 'O'), ('renowned', 'O'), ('scientist', 'O'), (',', 'O'), ('discussed', 'O'), ('climate', 'O'), ('change', 'O'), ('with', 'O'), ('president', 'O'), ('smith', 'B-PER'), ('at', 'O'), ('the', 'O'), ('e', 'B-LOC'), ('##iff', 'B-LOC'), ('##el', 'B-LOC'), ('tower', 'I-LOC'), (',', 'O'), ('emphasizing', 'O'), ('the', 'O'), ('need', 'O'), ('for', 'O'), ('urgent', 'O'), ('action', 'O'), ('.', 'O'), ('[SEP]', 'O')]
